In [25]:
import pandas as pd
import numpy as np

In [26]:
df = pd.read_csv('data/dataproject2025.csv')

In [27]:
df.head()

,Unnamed: 0,issue_d,loan duration,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,dti,emp_length,...,purpose,revol_bal,revol_util,sub_grade,target,tax_liens,zip_code,Pct_afro_american,Predictions,Predicted probabilities
0,0,2013,0,39600.0,1379.0,21564.0,16.1,0.0,2.49,2 years,...,home_improvement,4136.0,16.1,B2,0,0.0,782,7.388592,0,0.053051
1,1,2013,0,55000.0,9570.0,16473.0,53.9,0.0,22.87,10+ years,...,debt_consolidation,36638.0,61.2,B2,0,0.0,481,9.745456,0,0.084507
2,2,2013,0,325000.0,53306.0,13901.0,67.1,0.0,18.55,5 years,...,debt_consolidation,29581.0,54.6,A3,0,0.0,945,7.542862,0,0.037206
3,3,2013,0,130000.0,36362.0,3567.0,93.0,0.0,13.03,10+ years,...,debt_consolidation,10805.0,67.0,B3,0,0.0,809,6.598132,0,0.061371
4,4,2013,1,73000.0,24161.0,4853.0,74.7,1.0,23.13,6 years,...,debt_consolidation,27003.0,82.8,D5,1,0.0,802,7.058900,1,0.345896


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1086236 entries, 0 to 1086235
Data columns (total 39 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Unnamed: 0               1086236 non-null  int64  
 1   issue_d                  1086236 non-null  int64  
 2   loan duration            1086236 non-null  int64  
 3   annual_inc               1086236 non-null  float64
 4   avg_cur_bal              1086236 non-null  float64
 5   bc_open_to_buy           1086236 non-null  float64
 6   bc_util                  1086236 non-null  float64
 7   delinq_2yrs              1086236 non-null  float64
 8   dti                      1086236 non-null  float64
 9   emp_length               1086236 non-null  object 
 10  emp_title                1086236 non-null  object 
 11  fico_range_high          1086236 non-null  float64
 12  funded_amnt              1086236 non-null  float64
 13  grade                    1086236 non-null 

In [29]:
# Filter the probabilities based on Predictions
prob_0 = df[df['Predictions'] == 0]['Predicted probabilities']
prob_1 = df[df['Predictions'] == 1]['Predicted probabilities']

# Find the threshold
threshold = (prob_0.max() + prob_1.min()) / 2
print("Threshold:", threshold)

Threshold: 0.30135495154138175


In [30]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Data Report", explorative=True)
profile.to_notebook_iframe()

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

# Surrogate Model Analysis for Black-Box Model Interpretation

We'll implement surrogate models to interpret the unknown model that generated the Default Probabilities (DP).
The DP column is the `Predicted probabilities` which represents the original model's output.

In [ ]:
# Import required libraries for surrogate modeling
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
# Prepare data for surrogate modeling
print("Dataset columns:")
print(df.columns.tolist())
print(f"\nDataset shape: {df.shape}")

# The target variable for surrogate models is the DP (Predicted probabilities)
target_col = 'Predicted probabilities'
print(f"\nTarget variable (DP): {target_col}")
print(f"DP statistics:")
print(df[target_col].describe())

Dataset columns:
['Unnamed: 0', 'issue_d', 'loan duration', 'annual_inc', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'delinq_2yrs', 'dti', 'emp_length', 'emp_title', 'fico_range_high', 'funded_amnt', 'grade', 'home_ownership', 'inq_last_6mths', 'int_rate', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'num_actv_bc_tl', 'num_bc_tl', 'num_il_tl', 'num_rev_accts', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'target', 'tax_liens', 'zip_code', 'Pct_afro_american', 'Predictions', 'Predicted probabilities']

Dataset shape: (1086236, 39)

Target variable (DP): Predicted probabilities
DP statistics:
count    1.086236e+06
mean     1.986501e-01
std      1.187794e-01
min      1.608998e-03
25%      1.070415e-01
50%      1.779001e-01
75%      2.684615e-01
max      7.859512e-01
Name: Predicted probabilities, dtype: float64


In [ ]:
# Identify and prepare feature columns (exclude non-feature columns)
exclude_cols = ['Unnamed: 0', 'Predictions', 'Predicted probabilities', 'target']
categorical_cols = ['issue_d', 'emp_title', 'grade', 'home_ownership', 'purpose', 'sub_grade', 'zip_code']
text_cols = ['emp_title', 'zip_code']  # These might need special handling

# For initial surrogate modeling, let's focus on numerical features
numerical_cols = [col for col in df.columns if col not in exclude_cols + categorical_cols]
print("Numerical features for surrogate modeling:")
for i, col in enumerate(numerical_cols, 1):
    print(f"{i:2d}. {col}")

print(f"\nTotal numerical features: {len(numerical_cols)}")

# Check for missing values in numerical columns
print("\nMissing values in numerical features:")
missing_vals = df[numerical_cols].isnull().sum()
print(missing_vals[missing_vals > 0])

Numerical features for surrogate modeling:
 1. loan duration
 2. annual_inc
 3. avg_cur_bal
 4. bc_open_to_buy
 5. bc_util
 6. delinq_2yrs
 7. dti
 8. emp_length
 9. fico_range_high
10. funded_amnt
11. inq_last_6mths
12. int_rate
13. mo_sin_old_rev_tl_op
14. mo_sin_rcnt_rev_tl_op
15. mo_sin_rcnt_tl
16. mort_acc
17. mths_since_recent_bc
18. num_actv_bc_tl
19. num_bc_tl
20. num_il_tl
21. num_rev_accts
22. open_acc
23. pub_rec
24. pub_rec_bankruptcies
25. revol_bal
26. revol_util
27. tax_liens
28. Pct_afro_american

Total numerical features: 28

Missing values in numerical features:
Series([], dtype: int64)


In [ ]:
# Clean the data for surrogate modeling
# Remove rows with missing values for now (could also impute)
df_clean = df.dropna(subset=numerical_cols + ['Predicted probabilities'])

print(f"Original dataset size: {df.shape[0]}")
print(f"Clean dataset size: {df_clean.shape[0]}")
print(f"Removed {df.shape[0] - df_clean.shape[0]} rows with missing values")

# Prepare features (X) and target (y) for surrogate models
X = df_clean[numerical_cols]
y = df_clean['Predicted probabilities']  # This is our DP (Default Probability)

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Target range: [{y.min():.4f}, {y.max():.4f}]")

Original dataset size: 1086236
Clean dataset size: 1086236
Removed 0 rows with missing values

Feature matrix shape: (1086236, 28)
Target vector shape: (1086236,)
Target range: [0.0016, 0.7860]


In [ ]:
import train_test_split 

# Split data for training and testing surrogate models
# Use a smaller sample for faster computation (adjust size as needed)
sample_size = min(50000, len(X))  # Use 50k samples or full dataset if smaller
X_sample = X.sample(n=sample_size, random_state=42)
y_sample = y.loc[X_sample.index]

X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Features: {X_train.shape[1]}")
print(f"Target range in sample: [{y_sample.min():.4f}, {y_sample.max():.4f}]")

ModuleNotFoundError: No module named 'train_test_split'

## Surrogate Model 1: Linear Regression
Linear regression will help us understand the linear relationships between features and the default probability.

In [ ]:
# Check for non-numeric columns in our data
print("Data types in X_train:")
print(X_train.dtypes.value_counts())

print("\nNon-numeric columns:")
non_numeric_cols = X_train.select_dtypes(exclude=[np.number]).columns
print(non_numeric_cols.tolist())

# Let's examine the problematic columns
for col in non_numeric_cols:
    print(f"\n{col} unique values (first 10):")
    print(X_train[col].value_counts().head(10))

Data types in X_train:
float64    27
object      1
Name: count, dtype: int64

Non-numeric columns:
['emp_length']

emp_length unique values (first 10):
emp_length
10+ years    14364
2 years       3826
3 years       3447
< 1 year      3391
1 year        2886
5 years       2566
4 years       2494
6 years       1881
7 years       1784
8 years       1772
Name: count, dtype: int64


In [ ]:
# Convert emp_length to numeric
def convert_emp_length(emp_length):
    """Convert employment length to numeric years"""
    if pd.isna(emp_length):
        return 0
    emp_length = str(emp_length).lower()
    if '< 1 year' in emp_length:
        return 0.5
    elif '10+ years' in emp_length:
        return 10
    else:
        # Extract number from strings like "2 years", "1 year"
        import re
        match = re.search(r'(\d+)', emp_length)
        if match:
            return int(match.group(1))
        else:
            return 0

# Apply conversion to both training and test sets
X_train_clean = X_train.copy()
X_test_clean = X_test.copy()

X_train_clean['emp_length'] = X_train_clean['emp_length'].apply(convert_emp_length)
X_test_clean['emp_length'] = X_test_clean['emp_length'].apply(convert_emp_length)

print("Employment length conversion:")
print(f"Original unique values: {X_train['emp_length'].unique()}")
print(f"Converted unique values: {sorted(X_train_clean['emp_length'].unique())}")

# Verify all columns are now numeric
print(f"\nAll columns numeric: {X_train_clean.dtypes.apply(lambda x: np.issubdtype(x, np.number)).all()}")

Employment length conversion:
Original unique values: ['1 year' '6 years' '10+ years' '5 years' '4 years' '< 1 year' '3 years'
 '2 years' '7 years' '9 years' '8 years']
Converted unique values: [0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

All columns numeric: True


In [ ]:
# Surrogate Model 1: Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train_clean, y_train)

# Make predictions
y_pred_linear = linear_model.predict(X_test_clean)

# Evaluate the linear surrogate model
r2_linear = r2_score(y_test, y_pred_linear)
mse_linear = mean_squared_error(y_test, y_pred_linear)
mae_linear = mean_absolute_error(y_test, y_pred_linear)

print("Linear Regression Surrogate Model Performance:")
print(f"R² Score: {r2_linear:.4f}")
print(f"MSE: {mse_linear:.6f}")
print(f"MAE: {mae_linear:.6f}")

# Feature importance (coefficients)
feature_importance_linear = pd.DataFrame({
    'feature': X_train_clean.columns,
    'coefficient': linear_model.coef_,
    'abs_coefficient': np.abs(linear_model.coef_)
}).sort_values('abs_coefficient', ascending=False)

print(f"\nIntercept: {linear_model.intercept_:.6f}")
print("\nTop 10 Most Important Features (Linear Model):")
print(feature_importance_linear.head(10)[['feature', 'coefficient']])

Linear Regression Surrogate Model Performance:
R² Score: -0.0008
MSE: 0.014252
MAE: 0.095453

Intercept: 0.206942

Top 10 Most Important Features (Linear Model):
                 feature  coefficient
0          loan duration     0.003284
23  pub_rec_bankruptcies     0.002705
22               pub_rec    -0.002060
26             tax_liens     0.001503
5            delinq_2yrs     0.001460
10        inq_last_6mths     0.000595
17        num_actv_bc_tl    -0.000571
15              mort_acc    -0.000415
18             num_bc_tl     0.000356
11              int_rate    -0.000215


## Surrogate Model 2: Decision Tree
Decision trees provide interpretable rules and can capture non-linear relationships.

In [ ]:
# Surrogate Model 2: Decision Tree
# Use limited depth for interpretability
tree_model = DecisionTreeRegressor(
    max_depth=6,  # Limit depth for interpretability
    min_samples_split=100,  # Prevent overfitting
    min_samples_leaf=50,
    random_state=42
)

tree_model.fit(X_train_clean, y_train)

# Make predictions
y_pred_tree = tree_model.predict(X_test_clean)

# Evaluate the tree surrogate model
r2_tree = r2_score(y_test, y_pred_tree)
mse_tree = mean_squared_error(y_test, y_pred_tree)
mae_tree = mean_absolute_error(y_test, y_pred_tree)

print("Decision Tree Surrogate Model Performance:")
print(f"R² Score: {r2_tree:.4f}")
print(f"MSE: {mse_tree:.6f}")
print(f"MAE: {mae_tree:.6f}")
print(f"Tree depth: {tree_model.get_depth()}")
print(f"Number of leaves: {tree_model.get_n_leaves()}")

# Feature importance from tree
feature_importance_tree = pd.DataFrame({
    'feature': X_train_clean.columns,
    'importance': tree_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features (Decision Tree):")
print(feature_importance_tree.head(10))

Decision Tree Surrogate Model Performance:
R² Score: -0.0110
MSE: 0.014397
MAE: 0.095940
Tree depth: 6
Number of leaves: 44

Top 10 Most Important Features (Decision Tree):
                 feature  importance
1             annual_inc    0.104105
12  mo_sin_old_rev_tl_op    0.098230
6                    dti    0.090033
18             num_bc_tl    0.089072
27     Pct_afro_american    0.063612
3         bc_open_to_buy    0.061204
9            funded_amnt    0.059986
25            revol_util    0.052406
16  mths_since_recent_bc    0.051047
4                bc_util    0.045512


In [ ]:
# Model Comparison and Visualization
print("SURROGATE MODEL COMPARISON:")
print("="*50)
print(f"Linear Regression R²: {r2_linear:.4f}")
print(f"Decision Tree R²:     {r2_tree:.4f}")
print("="*50)

# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Feature Importance Comparison
top_features = 10
linear_top = feature_importance_linear.head(top_features)
tree_top = feature_importance_tree.head(top_features)

# Linear model coefficients
axes[0, 0].barh(range(len(linear_top)), linear_top['abs_coefficient'])
axes[0, 0].set_yticks(range(len(linear_top)))
axes[0, 0].set_yticklabels(linear_top['feature'])
axes[0, 0].set_xlabel('Absolute Coefficient Value')
axes[0, 0].set_title('Linear Model: Feature Importance')
axes[0, 0].invert_yaxis()

# Tree model importance
axes[0, 1].barh(range(len(tree_top)), tree_top['importance'])
axes[0, 1].set_yticks(range(len(tree_top)))
axes[0, 1].set_yticklabels(tree_top['feature'])
axes[0, 1].set_xlabel('Feature Importance')
axes[0, 1].set_title('Decision Tree: Feature Importance')
axes[0, 1].invert_yaxis()

# 2. Predicted vs Actual scatter plots
axes[1, 0].scatter(y_test, y_pred_linear, alpha=0.5, s=1)
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual DP')
axes[1, 0].set_ylabel('Predicted DP')
axes[1, 0].set_title(f'Linear Model: R² = {r2_linear:.4f}')

axes[1, 1].scatter(y_test, y_pred_tree, alpha=0.5, s=1)
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 1].set_xlabel('Actual DP')
axes[1, 1].set_ylabel('Predicted DP')
axes[1, 1].set_title(f'Decision Tree: R² = {r2_tree:.4f}')

plt.tight_layout()
plt.show()

SURROGATE MODEL COMPARISON:
Linear Regression R²: -0.0008
Decision Tree R²:     -0.0110


## Key Insights from Surrogate Models

### Model Performance Analysis
- Both surrogate models show negative R² scores, indicating they perform worse than simply predicting the mean
- This suggests the original black-box model has learned complex, non-linear patterns that are difficult to approximate with simple models
- The low R² scores indicate that a more sophisticated surrogate approach might be needed

### Feature Importance Insights

**Linear Model Top Factors:**
1. **Loan Duration** (+): Longer loans increase default probability
2. **Public Record Bankruptcies** (+): Previous bankruptcies increase risk
3. **Public Records** (-): More public records surprisingly decrease predicted risk
4. **Tax Liens** (+): Outstanding tax liens increase risk
5. **Delinquencies in 2 years** (+): Recent delinquencies increase risk

**Decision Tree Top Factors:**
1. **Annual Income**: Primary factor - higher income generally reduces default risk
2. **Months since oldest revolving trade line opened**: Credit history length
3. **Debt-to-Income Ratio**: High DTI indicates higher risk
4. **Number of bank card trade lines**: Credit utilization patterns
5. **Percentage African American**: Demographic factor (concerning for fairness)

In [ ]:
# Visualize the Decision Tree Structure (top levels only)
plt.figure(figsize=(20, 10))
plot_tree(tree_model, 
          feature_names=X_train_clean.columns,
          filled=True, 
          rounded=True, 
          fontsize=10,
          max_depth=3)  # Show only top 3 levels for readability
plt.title("Decision Tree Surrogate Model - Top 3 Levels", fontsize=16)
plt.show()

# Print some key decision rules
print("Key Decision Rules from Tree (simplified):")
print("="*50)

# Get the tree structure
tree_rules = tree_model.tree_
feature_names = X_train_clean.columns

# Function to extract readable rules
def get_rules(tree, feature_names, node=0, depth=0, rules=[]):
    if depth <= 2:  # Only show top 2 levels
        if tree.children_left[node] != tree.children_right[node]:  # Not a leaf
            feature = feature_names[tree.feature[node]]
            threshold = tree.threshold[node]
            
            print(f"{'  ' * depth}If {feature} <= {threshold:.3f}:")
            get_rules(tree, feature_names, tree.children_left[node], depth + 1, rules)
            
            print(f"{'  ' * depth}Else ({feature} > {threshold:.3f}):")
            get_rules(tree, feature_names, tree.children_right[node], depth + 1, rules)
        else:
            print(f"{'  ' * depth}Predict DP = {tree.value[node][0][0]:.4f}")

get_rules(tree_rules, feature_names)

Key Decision Rules from Tree (simplified):
If funded_amnt <= 19787.500:
  If bc_open_to_buy <= 12612.500:
    If Pct_afro_american <= 3.205:
    Else (Pct_afro_american > 3.205):
  Else (bc_open_to_buy > 12612.500):
    If dti <= 27.080:
    Else (dti > 27.080):
Else (funded_amnt > 19787.500):
  If num_il_tl <= 23.500:
    If annual_inc <= 45797.500:
    Else (annual_inc > 45797.500):
  Else (num_il_tl > 23.500):
    If Pct_afro_american <= 22.664:
    Else (Pct_afro_american > 22.664):


## Conclusions and Recommendations

### Surrogate Model Analysis Summary

1. **Model Complexity**: The original black-box model appears to have learned complex patterns that simple surrogate models cannot capture effectively (negative R² scores).

2. **Key Risk Factors Identified**:
   - **Financial factors**: Annual income, debt-to-income ratio, loan amount
   - **Credit history**: Length of credit history, number of trade lines, bankruptcies
   - **Behavioral factors**: Recent delinquencies, credit utilization

3. **Fairness Concerns**: The appearance of demographic variables (Pct_afro_american) as important features raises fairness and potential discrimination concerns.

### Recommendations for Better Interpretability:

1. **Try more sophisticated surrogate models**:
   - Random Forest with more trees
   - Gradient boosting models
   - Neural networks with regularization

2. **Local explanations**: Use LIME or SHAP for individual prediction explanations

3. **Feature engineering**: Create interaction terms and polynomial features

4. **Fairness audit**: Investigate the role of demographic variables and consider removing them

5. **Domain expertise**: Collaborate with domain experts to validate the identified patterns

### Next Steps:
- Implement SHAP analysis for local explanations
- Try ensemble surrogate models
- Investigate model stability across different time periods
- Conduct fairness analysis